In [1]:
%pip install ortools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [ ]:
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)

In [62]:
for n in (n[1] for n in nurses):
  print(n)

1
2
3
4
5


In [59]:
from ortools.sat.python import cp_model

# Problem data
nurses = [("Nurse 1", 1), ("Nurse 2", 2), ("Nurse 3", 3), ("Nurse 4", 4), ("Nurse 5", 5)]
num_nurses = range(5)
days = range(7)
shifts = range(3)
patient_density = [
    [1,3,5],
    [2,5,3],
    [1,4,2],
    [5,2,3],
    [2,4,1],
    [1,3,2],
    [2,4,2],
]

def suff(nurse_exp, patient_density):
    return nurse_exp >= patient_density

model = cp_model.CpModel()

shifts_worked = {}
for n in nurses:
    for d in days:
        for s in shifts:
            shifts_worked[(n[0], d, s)] = model.NewBoolVar(f"shift_{n[0]}_{d}_{s}")

# Constraints
for d in days:
    for s in shifts:
        model.AddExactlyOne(shifts_worked[(n[0], d, s)] for n in nurses)

for d in days:
    for s in shifts:
        model.AddAtMostOne(shifts_worked[(n[0], d, s)] for n in nurses)  # Each shift has at most one nurse assigned

for n in (n[0] for n in nurses): ####EN ÖNEMLİ Constraints
    for d in days:
        model.AddAtMostOne(shifts_worked[(n, d, s)] for s in shifts)

for d in days:
    for s in shifts:
        model.Add(sum(
            shifts_worked[(n[0], d, s)]
            for n in nurses if suff(n[1], patient_density[d][s])
        ) >= 1)
model.Maximize(
    sum(
        shifts_worked[(n[0], d, s)] * n[1] * (patient_density[d][s])
        for n in nurses
        for d in days
        for s in shifts
    )
)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print("Optimal schedule:")
    for d in days:
        print("Day", d)
        for s in shifts:
            for n in nurses:
                if solver.Value(shifts_worked[(n[0], d, s)]) == 1:
                    print(f"{n[0]} works shift {s}")
else:
    print("No optimal solution found.")


Optimal schedule:
Day 0
Nurse 3 works shift 0
Nurse 4 works shift 1
Nurse 5 works shift 2
Day 1
Nurse 3 works shift 0
Nurse 5 works shift 1
Nurse 4 works shift 2
Day 2
Nurse 3 works shift 0
Nurse 5 works shift 1
Nurse 4 works shift 2
Day 3
Nurse 5 works shift 0
Nurse 3 works shift 1
Nurse 4 works shift 2
Day 4
Nurse 4 works shift 0
Nurse 5 works shift 1
Nurse 3 works shift 2
Day 5
Nurse 3 works shift 0
Nurse 5 works shift 1
Nurse 4 works shift 2
Day 6
Nurse 4 works shift 0
Nurse 5 works shift 1
Nurse 3 works shift 2
